In [1]:
import sys
import logging
import datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pathlib

# nom_logs = os.path.basename("presentation")
# if os.path.isfile(f'{dir_logs}/{nom_logs}.log'):
#     shutil.move(f'{dir_logs}/{nom_logs}.log', f'{dir_logs}/{nom_logs}_old.log')


logger = logging.getLogger()
logger.setLevel(logging.INFO)

logFormatter = logging.Formatter("%(asctime)s [%(levelname)-5.5s]  %(message)s")

# # Handler pour écrire les logs dans un fichier
# fileHandler = logging.FileHandler(f"{dir_logs}/{nom_logs}.log")
# fileHandler.setFormatter(logFormatter)
# logger.addHandler(fileHandler)

# Handler pour afficher les logs dans le terminal
streamHandler = logging.StreamHandler(stream=sys.stdout)
streamHandler.setFormatter(logFormatter)
logger.addHandler(streamHandler)


# logger.info(f"Répertoire racine du projet : {p}")
# logger.info(f"Répertoire des logs : {dir_logs}")

In [2]:
##################################################################################################
#               Match de foot
##################################################################################################


START = dt.datetime.strptime("2018-01-01", "%Y-%m-%d")
STOP_DATA = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
# END = dt.datetime.strptime("2021-08-31", "%Y-%m-%d")
logger.info("On s'intéresse aux matchs de foot")
duree = 5
logger.info(f"  - On considère que l'impact d'un match est de {duree}h")

2024-05-28 11:07:04,738 [INFO ]  On s'intéresse aux matchs de foot
2024-05-28 11:07:04,739 [INFO ]    - On considère que l'impact d'un match est de 5h


In [ ]:
# https://www.ligue2.fr/clubs/Calendar?id=dijon-fco
# https://www.ligue2.fr/clubs/Calendar?id=fc-sochaux-montbeliard
def getDateAutherClubs(club, lg, lastDateYear):
    cptSaison = 0
    dateDispo = []
    saisons = []
    urlvr = "https://www.ligue" + lg + ".fr/clubs/Calendar?id=" + club
    try:
        r = requests.get(urlvr)
        soup = BeautifulSoup(r.text, 'html.parser')
        each_div = soup.findAll('div', {'class': 'CustomSelect-season'})[2:]
        for div in each_div:
            for el in div.findAll('li', {'class': 'CustomSelect-seasonItem'}):
                if cptSaison < 4:
                    cptSaison += 1
                    continue

                if lastDateYear == '':
                    saisons.append(el.findAll('a')[0]['href'])
                else:
                    if (int(lastDateYear) <= int(el.findAll('a')[0].text.split('/')[0])
                            or int(lastDateYear) <= int(el.findAll('a')[0].text.split('/')[1])):
                        saisons.append(el.findAll('a')[0]['href'])
    except:
        logger.warning("Problème avec la Ligue")
        # with open(dir_logs / 'probleme_de_connexion.csv', 'a') as f:
        #     f.write(f'{dt.datetime.now()},Foot2\n')
    for saison in saisons:
        try:
            r = requests.get(saison)
            soup = BeautifulSoup(r.text, 'html.parser')
            each_div = soup.findAll('div', {'class': 'CalendarClub-dateTime'})
            for cal in each_div:
                var = ''
                for temps in cal.findAll('span'):
                    var += temps.text.strip() + ' '
                if 'Reporté' in var or '--:--' in var:
                    continue
                dateDispo.append(dt.datetime.strptime(var.strip(), '%d/%m/%Y %Hh%M').strftime('%d/%m/%Y 00:00:00'))
        except:
            logger.warning("Problème de récupération des saisons de foot")
            # with open(dir_logs / 'probleme_de_connexion.csv', 'a') as f:
            #     f.write(f'{dt.datetime.now()},Foot3\n')
    return dateDispo

In [ ]:
def downloadArchiveFoot(numcomp, competitionName):
    dateDispo = []
    tabTotal = []
    now = dt.datetime.now()
    now = now.strftime('%Y-%m-%d 00:00:00')
    comps = ['france/ligue-1/', 'europe/ligue-des-champions/']
    urlvr = "https://www.footendirect.com/football/" + comps[numcomp]
    r = requests.get(urlvr)
    soup = BeautifulSoup(r.text, 'html.parser')
    res = soup.findAll('ul', {'class': 'dropdown-select'})
    date = START - dt.timedelta(days=1) #dt.datetime(2014, 12, 31, 23, 0, 0)
    if len(res) > 0:
        each_ul = res[0]
        saisons = []
        phases = [11, 24]
        if numcomp == 1:
            phases = [1, 2, 9]
        elif numcomp == 2:
            phases = [2, 9]
        for each_a in each_ul.findAll('a'):
            if each_a.text == '2013/2014' or each_a.text == '2014':
                break
            else:
                saisons.append(each_a['href'])
        for elt in reversed(saisons):
            urlvr = "https://www.footendirect.com" + elt
            r = requests.get(urlvr)
            soup = BeautifulSoup(r.text, 'html.parser')
            res = soup.findAll('ul', {'class': 'dropdown-select'})
            if len(res) >= 2:
                for ph in phases:
                    urlvr = "https://www.footendirect.com" + elt + "&competitionStageType1Id=" + str(
                        ph) + "&competitionRoundId=2"
                    r = requests.get(urlvr)
                    soup = BeautifulSoup(r.text, 'html.parser')
                    res1 = soup.findAll('table', {'class': 'table-condensed'})
                    if len(res1) > 0:
                        each_table = res1[0]
                        for tr in each_table.findAll('tr', attrs={'data-date': True}):
                            dateDispo.append(
                                dt.datetime.strptime(tr['data-date'], '%Y-%m-%d %H:%M').strftime('%d/%m/%Y 00:00:00'))
            else:
                logger.warning("Foot : Le retour de findall n'avait pas assez d'éléments")
            urlvr = "https://www.footendirect.com" + elt + "&competitionRoundId=" + "&competitionStageType1Id=24"
            r = requests.get(urlvr)
            soup = BeautifulSoup(r.text, 'html.parser')
            res2 = soup.findAll('table', {'class': 'table-condensed'})
            if len(res2) > 0:
                each_table = res2[0]
                for tr in each_table.findAll('tr', attrs={'data-date': True}):
                    dateDispo.append(
                        dt.datetime.strptime(tr['data-date'], '%Y-%m-%d %H:%M').strftime('%d/%m/%Y 00:00:00'))
    # recherche des matchs des clubs dijon et montbeliard dans les deux ligues (1 et 2)
    if numcomp == 0:
        dtDijon = getDateAutherClubs('dijon-fco', '1', '') + getDateAutherClubs('dijon-fco', '2', '')
        dtmontb = getDateAutherClubs('fc-sochaux-montbeliard', '1', '') + getDateAutherClubs('fc-sochaux-montbeliard',
                                                                                             '2', '')
        dtFinal = dtDijon + dtmontb
        dtFinal = removeDuplicatedDate(dtFinal)
        dateDispo = dateDispo + dtFinal
    while now != date.strftime('%Y-%m-%d 00:00:00'):
        tab = []
        date += dt.timedelta(days=1)
        dateVer = date.strftime('%Y-%m-%d')
        tab.append(dateVer)
        if dateVer in dateDispo:
            tab.append('1')
        else:
            tab.append('0')
        tabTotal.append(tab)
    df = pd.DataFrame(tabTotal, columns=['date_entree', competitionName])
    df.to_csv(dir_foot / (competitionName + ".csv"), index=False, date_format='%Y-%m-%d')
    return df

In [ ]:
def updateVariablesFoot(dataArchive, numcomp, competitionName):
    
    lastDate = dt.datetime.strptime(dataArchive.iloc[-1].date_entree, '%Y-%m-%d')

    lastDateYear = lastDate.year
    comps = ['france/ligue-1/', 'europe/ligue-des-champions/']
    urlvr = "https://www.footendirect.com/football/" + comps[numcomp]
    saisons = []
    phases = [11, 24]
    if numcomp == 1:
        phases = [1, 2, 9]
    elif numcomp == 2:
        phases = [2, 9]
    try:
        r = requests.get(urlvr)
        soup = BeautifulSoup(r.text, 'html.parser')
        each_ul = soup.findAll('ul', {'class': 'dropdown-select'})[0]
        for each_a in each_ul.findAll('a'):
            if ((numcomp != 2 and (int(lastDateYear) <= int(each_a.text.split('/')[0])
                                    or int(lastDateYear) <= int(each_a.text.split('/')[1])))
                    or (numcomp == 2 and int(lastDateYear) <= int(each_a.text))):
                saisons.append(each_a['href'])
    except:
        logger.error("soup.findAll n'a pas d'éléments")
    dateDispo = []
    date = dt.datetime(int(lastDateYear - 1), 12, 31, 0, 0, 0)
    now = dt.datetime.now()
    now = now.strftime('%Y-%m-%d 00:00:00')
    tabTotal = []
    for elt in reversed(saisons):
        urlvr = "https://www.footendirect.com" + elt
        try:
            # r = requests.get(urlvr)
            # soup = BeautifulSoup(r.text, 'html.parser')
            # each_ul = soup.findAll('ul', {'class': 'dropdown-select'})[1]
            for ph in phases:
                urlvr = "https://www.footendirect.com" + elt + "&competitionStageType1Id=" + str(
                    ph) + "&competitionRoundId=2"
                r = requests.get(urlvr)
                soup = BeautifulSoup(r.text, 'html.parser')
                res = soup.findAll('table', {'class': 'table-condensed'})
                if len(res) > 0:
                    each_table = res[0]
                    for tr in each_table.findAll('tr', attrs={'data-date': True}):
                        dateDispo.append(
                            dt.datetime.strptime(tr['data-date'], '%Y-%m-%d %H:%M').strftime('%d/%m/%Y 00:00:00'))
            urlvr = "https://www.footendirect.com" + elt + "&competitionRoundId=" + "&competitionStageType1Id=24"
            r = requests.get(urlvr)
            soup = BeautifulSoup(r.text, 'html.parser')
            each_table = soup.findAll('table', {'class': 'table-condensed'})[0]
            for tr in each_table.findAll('tr', attrs={'data-date': True}):
                dateDispo.append(dt.datetime.strptime(tr['data-date'], '%Y-%m-%d %H:%M').strftime('%d/%m/%Y 00:00:00'))
        except:
            logging.error('Problème de téléchargement footendirect.com')
            with open(dir_logs / 'probleme_de_connexion.csv', 'a') as f:
                f.write(f'{dt.datetime.now()},Foot1\n')
    if False: #numcomp == 0:
        dtDijon = getDateAutherClubs('dijon-fco', '1', lastDateYear) + getDateAutherClubs('dijon-fco', '2',
                                                                                          lastDateYear)
        dtmontb = getDateAutherClubs('fc-sochaux-montbeliard', '1', lastDateYear) + getDateAutherClubs(
            'fc-sochaux-montbeliard', '2', lastDateYear)
        dtFinal = dtDijon + dtmontb
        dtFinal = removeDuplicatedDate(dtFinal)
        dateDispo = dateDispo + dtFinal
    print(now, date.strftime('%Y-%m-%d 00:00:00'))
    while now != date.strftime('%Y-%m-%d 00:00:00'):
        tab = []
        date += dt.timedelta(days=1)
        dateVer = date.strftime('%Y-%m-%d')
        tab.append(dateVer)
        if dateVer in dateDispo:
            tab.append('1')
        else:
            tab.append('0')
            tabTotal.append(tab)
    df = pd.DataFrame(tabTotal, columns=['date_entree', competitionName])
    print(competitionName)
    datatFinal = pd.concat(
        [dataArchive[:dataArchive[dataArchive["date_entree"] == str(lastDateYear) + "-01-01"].index[0] + 1],
         df[df[df["date_entree"] == str(lastDateYear) + "-01-01"].index[0] + 1:]])
    datatFinal.to_csv(dir_foot / (competitionName + ".csv"), index=False)
    return datatFinal

In [ ]:
def removeDuplicatedDate(dtfi):
    unique_dates = []
    for fi in dtfi:
        if fi not in unique_dates:
            unique_dates.append(fi)
    return unique_dates

In [ ]:
dir_foot = pathlib.Path("__file__").resolve().parents[0]
dico_foot = {
    'LGF1': {'nom': 'Ligue 1',
             'code': 0},
    'CL': {'nom': 'Ligue des champions',
           'code': 1}
}

df0 = None
for foot in dico_foot:
    file_name = dir_foot / f"{foot}.csv"
    if not file_name.is_file():
        logger.info(f"  - on récupère l'archive de la {dico_foot[foot]['nom']}")
        downloadArchiveFoot(dico_foot[foot]['code'], foot)
    logger.info(f"  - On met à jour l'archive de la {dico_foot[foot]['nom']}")
    dataUpdateFoot = pd.read_csv(dir_foot / f"{foot}.csv", sep=',')
    
    df = updateVariablesFoot(dataUpdateFoot, dico_foot[foot]['code'], foot)
    if df0 is None:
        df0 = pd.DataFrame(index=df['date_entree'])
    df.set_index("date_entree", inplace=True)
    for k in range(duree):
        df[f"{foot}-{k}"] = df[foot].shift(k)
    df = df.bfill()[[f"{foot}-{k}" for k in range(duree)]].astype(int)
    df[f'match_{foot}'] = df[[f"{foot}-{k}" for k in range(duree)]].sum(axis=1)
    df = df[f'match_{foot}']
    df0 = pd.merge(df0, df, left_index=True, right_index=True)

logger.info(f"  - On crée les variables pour les jours à venir")
for col in df0.columns:
    for k in range(2, 13, 2):
        df0[f"{col}-{k}"] = df0[col].shift(-k)
    df0 = df0.copy()
    #for l in range(15):
    #    df0[f'{col}-av_{l}'] = df0.ffill()[[f"{col}-{k}" for k in range(1, 13)]].shift(12 * l).sum(axis=1)

#logger.info(f"  - On enregistre le csv foot_all.csv")
#df0.to_csv(dir_foot / 'foot_all.csv')

df0.reset_index(inplace=True)
try:
    df0['date_entree'] = pd.to_datetime(df0['date_entree'])
except:
    df0['date_entree'] = pd.to_datetime(df0['date_entree'], format="%d/%m/%Y %H:%M:%S")

df0 = df0.loc[df0['date_entree'] <= STOP_DATA]
df0 = df0.loc[START <= df0['date_entree']]

In [ ]:
# res = res[0]

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
current_date = dt.datetime.strptime("2023-08-20", "%Y-%m-%d")
# current_date = START
matches = []
competitions = []
matches_df = pd.DataFrame()
CM = pd.DataFrame()
while current_date <= STOP_DATA:
    logger.info(f"  - On récupère les matchs du {current_date.strftime('%d/%m/%Y')}")
    # Initialize lists to store match and competition information

    r = requests.get(f"https://www.footendirect.com/football/international/?selectedDate={current_date.strftime('%Y-%m-%d')}")
    soup = BeautifulSoup(r.text, 'html.parser')

    # Find the table containing the matches
    res = soup.find('table', {'class': "table matches table-condensed layout-fixed linked-rows"})

    # Find all rows in the table
    rows = res.findAll("tr")

    # Initialize variable to keep track of current competition
    current_competition = None

    # Process each row
    for row in rows:
        # Check if the row represents a competition badge
        if "competition-badge" in row.get("id", ""):
            competition_link = row.find("a", href=True)
            if competition_link:
                current_competition = competition_link.text.strip()
                competitions.append(current_competition)
        else:

            if row.get("class") and "league-data" in row.get("class") and "league-data-heading" not in row.get("class"):
                current_competition = row.find("div", class_="title").text.strip().split('\n')[1].strip()
                competitions.append(current_competition)

            # Check if the row represents a match
            match_data = row.find("td", class_="teams")
            if match_data:
                match_date = row["data-date"].split(" ")[0]
                match_time = row.find("td", class_="hour").text.strip()
                match_teams = match_data.findAll("a")
                match_teams = [team.text.strip() for team in match_teams][0]

                matches.append({"Date": match_date, "Time": match_time, "Competition": current_competition, "Teams": match_teams})

    matches_of_day = pd.DataFrame(matches)
    # Create DataFrame for matches
    matches_df = pd.concat([matches_df, matches_of_day])

    # # Print matches and competitions
    # print("Matches:")
    # print(matches_df)
    # print("\nCompetitions:")
    # print(competitions)
    if len(matches_of_day) > 0:
        CM = pd.concat([CM, pd.DataFrame([len(matches_of_day)], index=[current_date])], axis=0)
    current_date += dt.timedelta(days=1)

2024-05-28 11:36:20,490 [INFO ]    - On récupère les matchs du 20/08/2023
2024-05-28 11:36:21,763 [INFO ]    - On récupère les matchs du 21/08/2023
2024-05-28 11:36:23,613 [INFO ]    - On récupère les matchs du 22/08/2023
2024-05-28 11:36:25,968 [INFO ]    - On récupère les matchs du 23/08/2023
2024-05-28 11:36:27,699 [INFO ]    - On récupère les matchs du 24/08/2023
2024-05-28 11:36:29,733 [INFO ]    - On récupère les matchs du 25/08/2023
2024-05-28 11:36:32,113 [INFO ]    - On récupère les matchs du 26/08/2023
2024-05-28 11:36:34,362 [INFO ]    - On récupère les matchs du 27/08/2023
2024-05-28 11:36:36,502 [INFO ]    - On récupère les matchs du 28/08/2023
2024-05-28 11:36:38,395 [INFO ]    - On récupère les matchs du 29/08/2023
2024-05-28 11:36:40,436 [INFO ]    - On récupère les matchs du 30/08/2023
2024-05-28 11:36:42,307 [INFO ]    - On récupère les matchs du 31/08/2023
2024-05-28 11:36:44,279 [INFO ]    - On récupère les matchs du 01/09/2023
2024-05-28 11:36:46,204 [INFO ]    - O

In [8]:
matches_df.to_csv('matches4.csv')

In [91]:
loaded_df = pd.DataFrame()
for i in range(1, 5):
    matches_loaded = pd.read_csv(f"matches{i}.csv", index_col=0)
    loaded_df = pd.concat([loaded_df, matches_loaded])
loaded_df.drop_duplicates(inplace=True)

In [92]:
loaded_df['Date'] = pd.to_datetime(loaded_df['Date'])
result_df = loaded_df.pivot_table(index='Date', columns='Competition', aggfunc='size', fill_value=0)
all_dates = pd.date_range(START, STOP_DATA)
result_df = result_df.reindex(all_dates, fill_value=0)

# Réinitialiser l'index pour obtenir un DataFrame
result_df = result_df.reset_index().rename(columns={'index': 'date_entree'})

result_df.set_index('date_entree', inplace=True)


result_df.to_csv('INTERNATIONAL.csv')